# Прогнозирование стоимости автомобиля по характеристикам


Образовательная платформа: SkillFactory

Специализация: Data Science

Группа: DST-37 и 38

Юнит 6. Проект 5: "Выбираем автомобиль правильно"


### Задача:

    Создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам для того, чтобы выявлять выгодные предложения (когда желаемая цена продавца ниже предсказанной рыночной цены).

### Метрика:

    MAPE (Mean Percentage Absolute Error) - средняя абсолютная ошибка в процентах

### Нужно:

    Составить train датасет - спарсить данные, либо найти готовый
    Обучить модель

### Плюс:

    Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
    Сгенерировать новые признаки
    Подгрузить еще больше данных
    Попробовать подобрать параметры модели
    Попробовать разные алгоритмы и библиотеки ML
    Сделать Ансамбль моделей, Blending, Stacking

### Этапы работы:

    Парсинг с авто.ру - Вадим, Евгений, Артём
    EDA, Feature Engineering - Артём, Вадим, Евгений
    Сравнение одиночных моделей - Артём, Вадим
    Стекинг - Вадим
    
    
#### В данном ноутбуке мы проводим обучение моделей и выбираем лучшую для предсказания.

#### Также в этом проекте мы использовали:

Ноутбук, через который пытались парсить: https://www.kaggle.com/artemskakun/sf-dst-car-price-prediction-autoruparser

Спарсенный датасет мы взяли у этой команды, потому что парсинг занимал очень много времени: https://www.kaggle.com/juliadeinego/data-car-sales

Ноутбук, в котором провели EDA: https://www.kaggle.com/artemskakun/sf-dst-car-price-prediction-datapreprocessing

In [ ]:
# импорт библиотек

import glob
import pandas as pd
import numpy as np
import json
import csv
from datetime import datetime
from ast import literal_eval
import pandas.api.types as at

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
from xgboost.sklearn import XGBRegressor
import xgboost as xgb

from tqdm import tqdm
from sklearn.base import clone

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)

RANDOM_SEED = 42

TEST_DATA = '../input/preproc/'
# TEST_DATA = 'D:/skillfactory_rds/CarPricePrediction/'
# TEST_DATA = 'data/test/'

### Функции

In [ ]:
# функция распределения признаков по типу данных
def sort_features(df_raw, target_cols, time_cols, num_cols, bin_cols, cat_cols, count_col):
    for col in df_raw.columns:
        if col in target_cols or col in time_cols or col in num_cols\
                or col in bin_cols or col in cat_cols or col in count_col:
            continue
        elif len(df_raw[col].value_counts()) == 1:
            df_raw.drop(columns=[col], inplace=True)
        elif at.is_datetime64_any_dtype(df_raw[col]):
            time_cols.append(col)
        elif at.is_numeric_dtype(df_raw[col]):
            if len(df_raw[col].value_counts()) == 2:
                bin_cols.append(col)
            else:
                num_cols.append(col)
        elif at.is_string_dtype(df_raw[col]):
            cat_cols.append(col)
        else:
            print(
                f'Столбец {col} не был причислен ни к одной категории\n'+'_'*50)

    print_cols_lists(df_raw, target_cols, time_cols,
                     num_cols, bin_cols, cat_cols, count_col)

    return target_cols, time_cols, num_cols, bin_cols, cat_cols, count_col

# функция вывода данных по типу
def print_cols_lists(df, target_cols, time_cols, num_cols, bin_cols, cat_cols, count_col):
    print('\nКлючевые признаки: ', target_cols)
    print('\nПризнаки даты или времени: ', time_cols)
    print('\nКатегориальные признаки: ', cat_cols)
    print('\nБинарные признаки: ', bin_cols)
    print('\nКоличественные признаки: ', num_cols)
    print('\nПризнаки-счетчики: ', count_col)
    print('\nВ датасете: строк - ', len(df), 'колонок - ', len(df.columns))

# рассчёт поправочного коэффициента в зависимости от изменение курса доллара
prev_rate = 77.9241
curr_rate = 76.9808  # 16/04/21
rate_coeff = prev_rate/curr_rate
print(f'поправочный коэффициент {rate_coeff}')
date_before = datetime.strptime('01/04/2021', '%d/%m/%Y')

# функция обучения модели и вывода MAPE
# is_log - передавать значение True, если y_train прологарифмирован
def learn_model(model, X_train, X_test, y_train, y_test, is_log=False):
    model.fit(X_train, y_train)
    y_pred = []
    if is_log:
        print('predict logarithmic')
        y_pred = np.round(np.exp(model.predict(X_test))).astype(int)
    else:
        print('predict native')
        y_pred = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    print(f"Средняя абсолютная ошибка в процентах: {mape*100:0.2f}%")
    return mape

# функция сохранения результатов в списке results
def safe_results(results, model_name, mape, test_prediction, sabmission):
    results[model_name] = {
        'mape': mape, 'test_prediction': test_prediction, 'submission': sabmission}

# функция получения предсказания. функция учитывает изменение курса доллара между временем парсинга обучаещего
# и тестового датасета через rate_coeff
def model_prediction(model, X_test, test_df, is_log=False):
    predict_test = []
    predict_submission = []
    if is_log:
        print('predict logarithmic')
        predict_test = np.round(np.exp(model.predict(X_test)), -3).astype(int)
        predict_submission = np.round(
            np.exp(model.predict(test_df))/rate_coeff, -3).astype(int)
    else:
        print('predict native')
        predict_test = np.round(model.predict(X_test), -3)
        predict_submission = np.round(model.predict(test_df)/rate_coeff, -3)

    return predict_test, predict_submission

# функция сохранения прогноза в файл
def write_submission_to_file(name, submission):
    submission = np.around(submission).astype(int)
    with open(f'{TEST_DATA}{name}.csv', mode='w') as submission_file:
        writer = csv.writer(submission_file, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(submission.tolist())

# функция подставления прогноза
def make_submission(predict_submission, version):
    test_submission = pd.read_csv(f'{TEST_DATA}sample_submission.csv')
    predict = np.around(predict_submission).astype(int)
    test_submission['price'] = predict

    test_submission.to_csv(
        f'{TEST_DATA}submission_v{version}.csv', index=False)

# Функция для определения границ выбросов
def get_outliners(column):
    koeff = 1.5
    median = column.median()
    quan25 = column.quantile(0.25)
    quan75 = column.quantile(0.75)
    IQR = quan75 - quan25
    left = quan25 - koeff*IQR,
    right = quan75 + koeff*IQR
    print(f"Границы выбросов для столбца '{column.name}': [{left}, {right}]")
    return(left, right)

### Загружаем предподготовленные данные

In [ ]:
full_df = pd.read_csv(f'{TEST_DATA}preproc.csv')
full_df.sample(2)

### Подготовим списки признаков по типам

In [ ]:
target_cols = ['price']
num_cols, bin_cols, cat_cols, time_cols, count_col = [], [], [], [], []

target_cols,time_cols,num_cols,bin_cols,cat_cols,count_col = sort_features(full_df
                                                                           ,target_cols,time_cols
                                                                           ,num_cols,bin_cols
                                                                           ,cat_cols,count_col)

### Оценка степени влияния признаков на целевые переменные

#### Посмотрим на распределение числовых данных

In [ ]:
for i in num_cols:
    plt.figure()
    sns.distplot(full_df[i][full_df[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

Распределение признаков нормальное.

Хотя количество владельцев и выглядит как категоральный признак, оставляем его числовым, т.к. количество владельцев явно влияет на цену автомобиля.

Количество дверей перенесём в категориальные признаки.

In [ ]:
num_cols.remove('number_of_doors')
cat_cols.append('number_of_doors')

#### Попробуем прологарифмировать колонки и проверим распределение на копии датафрейма

In [ ]:
full_df_copy = full_df.copy()
cols = ['engine_displacement', 'engine_power']
fig, axes = plt.subplots(1, len(cols), figsize=(10,7))
for i,col in enumerate(cols):
    full_df_copy[col] = np.log(full_df_copy[col] + 1)
    sns.distplot(full_df_copy[col][full_df_copy[col] > 0].dropna(), ax=axes.flat[i],kde = False, rug=False, color="b")

#### Попробуем взять квадратный корень тех же колонок и проверим распределение на копии датафрейма

In [ ]:
full_df_copy = full_df.copy()
cols = ['engine_displacement', 'engine_power']
fig, axes = plt.subplots(1, len(cols), figsize=(10,7))
for i,col in enumerate(cols):
    full_df_copy[col] = np.sqrt(full_df_copy[col])
    sns.distplot(full_df_copy[col][full_df_copy[col] > 0].dropna(), ax=axes.flat[i],kde = False, rug=False, color="b")

Логарифмирование признаков улучшает распределение

Прологарифмируем признаки engine_displacement и engine_power

In [ ]:
cols = ['engine_displacement', 'engine_power']
for i,col in enumerate(cols):
    full_df[col] = np.log(full_df[col] + 1)

#### Построим boxplot’ы для числовых переменных

In [ ]:
for i in num_cols:
    plt.figure()
    sns.boxplot(full_df[i][full_df[i] > 0].dropna())
    plt.title(i)
    plt.show()

# Обработка выбросов числовых признаков

In [ ]:
left, right = get_outliners(full_df['price'])
full_df[((full_df['price'] < left) | (full_df['price'] > right)) & (full_df['test'] == 0)]

In [ ]:
left, right = get_outliners(full_df['engine_power'])
full_df[((full_df['engine_power'] < left) | (full_df['engine_power'] > right)) & (full_df['test'] == 0)]

In [ ]:
left, right = get_outliners(full_df['mileage'])
full_df[((full_df['mileage'] < left) | (full_df['mileage'] > right)) & (full_df['test'] == 0)]

In [ ]:
left, right = get_outliners(full_df['model_date'])
full_df[((full_df['model_date'] < left) | (full_df['model_date'] > right)) & (full_df['test'] == 0)]

In [ ]:
full_df.info()

Явно видны выбросы на всех признаках, но они связаны с присутствием редких и дорогих моделей. Мы не будем от них отказываться, т.к. хотелось бы, чтобы модель умела предсказывать и их стоимость.
Выбросы по дате модели находятся в тестовой выборке, так что их тоже не будем трогать.
Тем не менее мы обучали модели на очищенной от выбросов выборке, но это не дало прироста качества моделей.

### Оценка корреляции числовых признаков

In [ ]:
cols = num_cols.copy()
cols.append('price')
sns.heatmap(full_df[full_df['test']==0][cols].corr().abs(), vmin=0, vmax=1, annot=True)

Мы видим сильную корреляцию всех признаков с целевым. 

Так же присутствует сильная корреляция между датой выпуска, пробегом и количеством владельцев, что и понятно.

### Оценим значимость числовых признаков

Посмотрим на коэффициент корреляции Пирсона между признаками и целевой переменной.

In [ ]:
X_train = full_df[full_df['test']==0].drop('price', axis = 1)
y_train = full_df[full_df['test']==0]['price']
correlations = X_train.corrwith(y_train).sort_values(ascending=False)
plot = sns.barplot(y=correlations.index, x=correlations)

График подтверждает реальную ситуацию: количество владельцев и пробег отрицательно влияют на стоимость автомобиля, а мощность, объем двигателя и более поздняя дата выпуска увеличивают ее.

#### Добавим новые признаки возведя в квадрат пробег, объем двигателя и его мощность для того, что бы усилить не линейное влияние этих признаков при обучении модели.

In [ ]:
for col in ['engine_displacement','engine_power','mileage']:
    name = col + '_x2'
    full_df[name] = full_df[col].pow(2)
    num_cols.append(name)

еще раз оценим корреляцию

In [ ]:
cols = num_cols.copy()
cols.append('price')
sns.heatmap(full_df[full_df['test']==0][cols].corr().abs(), vmin=0, vmax=1, annot=True)

In [ ]:
full_df.sample(2)

### Исследуем категориальные признаки

In [ ]:
for col in ['body_type','color','vehicle_transmission','vehicle_pasport','wheel','brand']:
    ax = sns.countplot(x=col, data=full_df)
    ax.xaxis.set_tick_params(rotation=45)
    plt.show()

In [ ]:
full_df[cat_cols].sample(2)

In [ ]:
full_df.model_name.value_counts()

Очень большое количество значений имеет имя модели.

Всвязи с большим разнообразием наименований моделей, преобразуем их.

1% наименее используемый обозначим как имя бренда.

In [ ]:
model_lst = []
for brand in full_df['brand'].unique():
    count_brand = full_df[full_df['brand'] == brand]['price'].count()
    for model in full_df[full_df['brand'] == brand]['model_name'].unique():
        count_model = full_df[full_df['model_name'] == model]['price'].count()
        if count_model/count_brand < 0.01:
            model_lst.append(model)

for model in model_lst:
    try:
        full_df.loc[full_df['model_name'] == model, 'model_name'] = full_df[full_df['model_name'] == model]['brand'].unique()[0]
    except:
        pass           

In [ ]:
full_df.model_name.value_counts()

Теперь количество моделей стало более приемлемым.

#### Для оценки значимости категориальных признаков, преобразуем их в числовые значения.

Для начала используем LabelEncoder для преобразования категориальных признаков в числовые значения, предварительно скопировав в новый датафрейм.

In [ ]:
df_copy = full_df[full_df['test']==0][cat_cols + target_cols]

label_encoder = LabelEncoder()

for column in cat_cols:
    df_copy[column] = label_encoder.fit_transform(df_copy[column])
    
# убедимся в преобразовании    
df_copy.sample(2)

#### Оценим значимость категориальных признаков

In [ ]:
X_train = df_copy.drop('price', axis = 1)
y_train = df_copy['price']

imp_cat = pd.Series(mutual_info_classif(X_train, y_train,\
                    discrete_features = True), index = X_train.columns)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Наибольшее влияние на стоимость оказывают модель, брэнд, тип кузова, цвет и тип привода.

### Машинное обучение

#### Преобразуем категоральные признаки в отдельные признаки.

In [ ]:
dummies = pd.get_dummies(full_df[cat_cols])
full_df = full_df.drop(cat_cols, axis=1).join(dummies)
full_df.sample(3)

#### Разделим обучающую выборку для обучения

In [ ]:
X = full_df[full_df['test']==0].drop(['price'], axis = 1)
X = X.drop('test', axis = 1)
y = full_df[full_df['test']==0]['price'].values

test_df = full_df[full_df['test'] == 1].drop(['test','price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42, test_size=0.2)

#### Проведем стандартизацию числовых признаков, предварительно отделив тестовую и обучающую выборку.

In [ ]:
scaller = StandardScaler()

X_train_transformed = X_train.copy()
X_train_transformed[num_cols] = scaller.fit_transform(X_train_transformed[num_cols])

X_test_transformed = X_test.copy()
X_test_transformed[num_cols] = scaller.transform(X_test_transformed[num_cols])

test_df_transformed = test_df.copy()
test_df_transformed[num_cols] = scaller.transform(test_df_transformed[num_cols])

y_train_log = np.log(y_train+1)


# Выборка, включающая все спарсиные данные. Будем применять ее для обучения перед submission
X_transformed = X.copy()
X_transformed[num_cols] = scaller.transform(X_transformed[num_cols])

y_log = np.log(y+1)

##### Результаты будем сохранять в словаре results

In [ ]:
results = {}
version = 0

#### Построим линейную регрессию с целой и логарифмированой целевой переменной

Комментируем все модели, представив их результаты.

In [ ]:
'''
lr = LinearRegression()
print ('Исследуем линейную регрессию');
version = 1

mape = learn_model(lr, X_transformed, X_test_transformed, y, y_test)
predict_test, predict_submission = model_prediction(lr, X_test_transformed, test_df_transformed)
safe_results(results, 'LinearRegression', mape, predict_test, predict_submission)
make_submission(predict_submission, version)


print ('Исследуем линейную регрессию с логарифмированием целевой переменной');
version = 2

mape = learn_model(lr, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(lr, X_test_transformed, test_df_transformed, True)
safe_results(results, 'LinearRegression_log', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 50.98%
# kagle 226916

# с логарифмированием целевой переменной
# Средняя абсолютная ошибка в процентах: 15.24%
# kagle 258490

Как и ожидалось, линейная регрессия абсолютно не справляется с задачей, всвязи с тем, что изменение цены нелинейно.

#### Обучим модель на стандартных настройках логистической регрессии.

In [ ]:
'''
print ('Исследуем логистическую регрессию');
version = 3

logreg = LogisticRegression(max_iter=100)

mape = learn_model(logreg, X_transformed, X_test_transformed, y, y_test)
predict_test, predict_submission = model_prediction(logreg, X_test_transformed, test_df_transformed)
safe_results(results, 'LogisticRegression', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 17.65%
# kagle 36.53207

#### Попробуем случайный лес

In [ ]:
'''
print ('Исследуем случайный лес');
version = 4

rf = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)

mape = learn_model(rf, X_transformed, X_test_transformed, y, y_test)
predict_test, predict_submission = model_prediction(rf, X_test_transformed, test_df_transformed)
safe_results(results, 'LogisticRegression', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 3.08%
# kagle 26.11943

'''
print ('Исследуем случайный лес с логарифмированием целевой переменной');
version = 5

mape = learn_model(rf, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(rf, X_test_transformed, test_df_transformed, True)
safe_results(results, 'LogisticRegression_log', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 2.93%
# kagle 19.03580

#### Подберем параметры для случайного леса

In [ ]:
'''
parameters = {'n_estimators': [200, 1100, 2000],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [10, 60, 110],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rf = RandomForestRegressor(random_state=RANDOM_SEED)
random_grid = RandomizedSearchCV(estimator=rf, param_distributions=parameters, n_iter=100, 
                               cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)
                               '''

In [ ]:
'''
print ('Исследуем случайный лес с гипер параметрами');
version = 6

est = random_grid.best_estimator_
rf_random = RandomForestRegressor(n_estimators=est.n_estimators, min_samples_split=2, min_samples_leaf=1, 
                             max_features=3, max_depth=25, bootstrap=True, random_state=RANDOM_SEED)

mape = learn_model(rf_random, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(rf_random, X_test_transformed, test_df_transformed, True)
safe_results(results, 'RandomForestRegressor_HP', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

Ни одна из наших машин не смогла прогнать RandomizedSearchCV

#### Попробуем ExtraTreesRegressor

In [ ]:
'''
print ('Исследуем ExtraTreesRegressor с подбором гиперпараметров');
version = 7
parameters = {'n_estimators' : [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)],
              'max_depth' : [3, 5, 7, 10, 15, None],
              'min_samples_split' : [2, 4, 6],
              'bootstrap' : [True, False]
             }

etr = ExtraTreesRegressor(random_state=RANDOM_SEED)
etr_grid = RandomizedSearchCV(estimator = etr,
                        param_distributions = parameters, 
                        cv = 3, 
                        verbose=2)
mape = learn_model(etr_grid, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(xgb_grid, X_test_transformed, test_df_transformed, True)
safe_results(results, 'ExtraTreesRegressorHP', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 2.84%
# kagle - 19.9

### Бустинг

#### Попробуем градиентный бустинг 

In [ ]:
'''
print ('Исследуем GradientBoosting');
version = 10

gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)

mape = learn_model(gb, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(xb, X_transformed, test_df_transformed, True)
safe_results(results, 'GradientBoostingRegressor', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 7.26%
# kagle 20.21

#### Попробуем XGBRegressor с L1 regularization

In [ ]:
'''
print ('Исследуем XGBRegressor с alpha=1.5 (L1 regularization)');
version = 11

xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, reg_alpha=1.5, n_jobs=-1, n_estimators=500)

mape = learn_model(xb, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(xb, X_transformed, test_df_transformed, True)
safe_results(results, 'XGBRegressorL1', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 7.96%
# kagle 18.5

Заметно что дополнительные признаки оказывают положительное влияние

#### Попробуем XGBRegressor с L2 regularization

In [ ]:
'''
print ('Исследуем XGBRegressor с lambda=1.5 (L2 regularization)');
version = 12

xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, reg_lambda=1.5, n_jobs=-1, n_estimators=500)
mape = learn_model(xb, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(xb, X_test_transformed, test_df_transformed, True)
safe_results(results, 'XGBRegressorL2', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 7.25%
# kagle 18.81

#### Попробуем CatBoostRegressor

In [ ]:
'''
from catboost import CatBoostRegressor
print ('Исследуем CatBoostRegressor')
version = 13

cbr = CatBoostRegressor(iterations=5000, learning_rate=1, depth=2, random_seed=RANDOM_SEED)

mape = learn_model(cbr, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(cbr, X_transformed, test_df_transformed, True)
safe_results(results, 'CatBoostRegressor', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 9.94%
# kagle 28.33416

#### Подберем параметры для CatBoostRegressor

In [ ]:
# Уже подобранные значения ниже:

# grid = {'learning_rate': [0.03, 0.1],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9]}
# cbr = CatBoostRegressor(iterations=100, learning_rate=1, depth=2, random_seed=RANDOM_SEED)
# grid_search_result = cbr.grid_search(grid, X=X_transformed, y=y_log, plot=True)

Лучшими параметрами выбраны: {'depth': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.1}

In [ ]:
# grid_search_result
# {'params': {'depth': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.1}

Результат модели с лучшими параметрами

In [ ]:
'''
print ('Исследуем CatBoostRegressor с параметрами GridSearch')
version = 14

cbr = CatBoostRegressor(iterations=2000,
                        learning_rate=0.1,
                        depth=10,
                        l2_leaf_reg=1,
                        random_seed=RANDOM_SEED)

mape = learn_model(cbr, X_transformed, X_test_transformed, y_log, y_test, True)
predict_test, predict_submission = model_prediction(cbr, X_transformed, test_df_transformed, True)
safe_results(results, 'CatBoostRegressor_GS', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 7.99%
# kagle 21.29833

**Примечение**

Дополнительно, чтобы улучшить качество моделей, мы пытались разделить все автомобили на 2 части (дорогие и дешевые). Для разделения использовали признак 'engine_power'. Обучали и предсказывали отдельно по каждому датасету. Результат соединяли в единый файл submission. Но в итоге значимого прироста качества не получили, поэтому далее представлен код для единого датасета

In [ ]:
# cheap = full_df[full_df['engine_power'] <= 250]
# expensive = full_df[full_df['engine_power'] > 250]
# expensive.to_csv(path_or_buf=f'{TEST_DATA}expensive.csv')
# cheap.to_csv(path_or_buf=f'{TEST_DATA}cheap.csv')

### Стекинг

#### Функции

In [ ]:
# рассчёт алгоритма
def compute_meta_feature(regr, X_train, X_test, y_train, test_df, cv):
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    

    splits = cv.split(X_train)
    print (splits)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)

    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)

    X_meta_test = meta_regr.predict(X_test)
    X_meta_pred = meta_regr.predict(test_df)

    return X_meta_train, X_meta_test, X_meta_pred

# рассчёт набора алгоритмов
def generate_meta_features(regr, X_train, X_test, y_train, test_df, cv):
    features = [compute_meta_feature(regr, X_train, X_test, y_train, test_df, cv) for regr in tqdm(regr)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test, features_pred in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test, features_pred in features]).T
    stacked_features_pred = np.vstack([features_pred for features_train, features_test, features_pred in features]).T
    return stacked_features_train, stacked_features_test, stacked_features_pred


#### Список моделей для стэкинга

In [ ]:
# выбраем наиболее успешные модели
models = [
RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
ExtraTreesRegressor(n_estimators=300, 
                    criterion='mse', 
                    bootstrap=True, 
                    n_jobs=-1, 
                    random_state=RANDOM_SEED
            ),
XGBRegressor(objective='reg:squarederror', 
                    n_estimators=300,
                    colsample_bytree=0.5, 
                    learning_rate=0.03,
                    max_depth=12, 
                    reg_alpha=1.5, 
                    n_jobs=-1
            ),
XGBRegressor(objective='reg:squarederror', 
                    n_estimators=300,
                    colsample_bytree=0.5, 
                    learning_rate=0.03,
                    max_depth=12, 
                    reg_lambda=1.5, 
                    n_jobs=-1
            )
]

##### Проведем обучение базовых моделей

In [ ]:
'''
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features(models
    , X_transformed.values, X_test_transformed.values, y_log, test_df_transformed.values, cv)
'''

#### Проведем обучение мета модели

Попробуем линейную регрессию в качестве мета модели

In [ ]:
'''
print ('Исследуем линейную регрессию');
version = 20

lr = LinearRegression()
mape = learn_model(lr, stacked_features_train, stacked_features_test, y_log, y_test, True)
predict_test, predict_submission = model_prediction(lr, stacked_features_test, stacked_features_pred, True)
safe_results(results, 'Stacking', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 4.65%
# kagle 18.58

Попробуем случайный лес в качестве мета модели

In [ ]:
'''
print ('Исследуем случайный лес');
version = 21

rf = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
mape = learn_model(rf, stacked_features_train, stacked_features_test, y_log, y_test, True)
predict_test, predict_submission = model_prediction(rf, stacked_features_test, stacked_features_pred, True)
safe_results(results, 'Stacking', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 5.23%
# kagle 19.52

#### Попробуем другой состав стекинга

In [ ]:
'''
models = [
    RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1),
    ExtraTreesRegressor(n_estimators=200, min_samples_split=2,
                        max_depth=None, bootstrap=True)
]

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features(
    models, X_transformed.values, X_test_transformed.values, y_log, test_df_transformed.values, cv)

print('Исследуем линейную регрессию')
version = 22

lr = LinearRegression()
mape = learn_model(lr, stacked_features_train,
                   stacked_features_test, y_log, y_test, True)
predict_test, predict_submission = model_prediction(
    lr, stacked_features_test, stacked_features_pred, True)
safe_results(results, 'Stacking', mape, predict_test, predict_submission)
make_submission(predict_submission, version)
'''

# Средняя абсолютная ошибка в процентах: 2.85%
# kagle 18.77

In [ ]:
'''
models = [
RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1),
ExtraTreesRegressor(n_estimators=300, 
                    criterion='mse', 
                    bootstrap=True, 
                    n_jobs=-1, 
                    random_state=RANDOM_SEED
            ),
XGBRegressor(objective='reg:squarederror', 
                    n_estimators=300,
                    colsample_bytree=0.5, 
                    learning_rate=0.03,
                    max_depth=12, 
                    reg_alpha=1.5, 
                    n_jobs=-1
            ),
KNeighborsRegressor(algorithm = 'brute', weights = 'distance', p=1
            )
]

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features(models
    , X_transformed.values, X_test_transformed.values, y_log, test_df_transformed.values, cv)
'''

# Средняя абсолютная ошибка в процентах: 6.44%
# kagle 19.06

## Результат работы:

По результатам обучения различных моделей, наилучший результат показала XGBRegressor с параметром reg_lambda=1.5

В работе удалось создать модель, которая предсказывает стоимость автомобиля по его характеристикам. Эту модель можно использовать для выявления выгодных предложений, когда желаемая цена продавца ниже предсказанной рыночной цены.

В ходе создания модели было сделано:
1. Составлен train датасет из данных с сайта auto.ru
2. На этих данных обучена модель
3. Дополнительно извлечены и сгенерированы новые признаки, в том числе с учетом изменения курса валюты
4. Подобрана оптимальные параметры модели
5. Попроботано большое количество разных алгоритмов и библиотек ML
6. Реализован Stacking

## Выводы:

Сильное различие в метриках на обучаемом и тестовом датасете могут быть связаны с различным периодом парсинга данных.

Большинство моделей (исключая линейную регрессию) показали близкие результаты. Дальнейшее улучшение прогноза можно попытаться достичь за счет подбора гипер параметров.

К сожалению, ни один из компьютеров не смог выполнить полный подбор гипер параметров за разумное время. В связи с этим заметно переобучение на обучаемом датасете.